In [166]:
import pandas as pd
import numpy as np
import requests
import os
import time
# import matplotlib.pyplot as plt

In [167]:
token = "dc76ceadd678dd146f345afcb88619b4ab0287e9"
headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Token ' + token
        }
requests.get("https://api.tiingo.com/api/test/",
                                    headers=headers)

<Response [200]>

In [168]:
d1 = dates[5]
d2 = dates_end[5] 

query = {'tickers':tickers,
             'startDate': d1, 
             'endDate': d2,
             'resampleFreq': '1hour'}

r = requests.get("https://api.tiingo.com/tiingo/crypto/prices", 
                 params = query,
                 headers=headers)

In [169]:
print(d1, d2)
r.json()

2017-12-30 00:00:00 2018-01-02 00:00:00


[]

In [170]:
import time
from datetime import datetime as dt
from datetime import timedelta

_from = dt.now()
to = _from + timedelta(hours=10, minutes=5)

In [171]:
class counter:
    def __init__(self, wait_h, wait_m, wait_s, period_max):
        self.wait_h = wait_h
        self.wait_m = wait_m
        self.wait_s = wait_s
        self.wait = timedelta(hours=self.wait_h, minutes=self.wait_m, seconds=self.wait_s)
        
        self.period_max = period_max
        
        self.start = dt.now()
        self.to = self.start + self.wait
        self.count = 0
        
    def add(self):
        self.last_add = dt.now()
        if self.last_add > self.start+self.wait:
            self.reset()
        
        self.count+=1
        
        if self.count == self.period_max:
            print('Reach maximum request in a period, wait until %s' %self.to)
            time.sleep((self.to - dt.now()).seconds)
            self.reset()
   
    def reset(self):
        self.start = dt.now()
        self.to = self.start +  self.wait
        self.count = 0
        print('counter reset')

In [152]:
_counter = counter(0,0,3,20)

for i in range(500):
    _counter.add()
    print(_counter.count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:43.722980
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:45.724464
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:47.725562
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:49.728198
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:51.733777
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:53.735406
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request, wait until 2021-07-07 12:55:55.740478
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum hourly request

KeyboardInterrupt: 

In [153]:
_counter.add()

counter reset


In [172]:
def get_price(tickers, dates, dates_end, _counter):
    error_log = []

    for d1, d2 in zip(dates, dates_end):
        d1 = str(d1).split(' ')[0]
        d2 = str(d2).split(' ')[0]
        print(d1, d2)

        query = {'tickers':tickers,
                 'startDate': d1, 
                 'endDate': d2,
                 'resampleFreq': '1hour'}

        
        r = requests.get("https://api.tiingo.com/tiingo/crypto/prices", 
                         params = query,
                         headers=headers)
        
        _counter.add()
        print('hourly request count:',_counter.count)

        if len(r.json())==0:
            error_log.append([(tickers, d1,d2)])
            print('Empty Json')

        else:
            df=pd.DataFrame(r.json()[0]['priceData'])
            df=df[['date', 'open', 'high', 'low', 'close', 'volume', 'tradesDone', 'volumeNotional']]
            df.to_csv(filename, mode="a", header=False)
        
    return error_log
#     time.sleep(0.5)          

In [173]:
def clean_data(filename):
    file = pd.read_csv(filename, header=None)
    file.columns = ['id', 'date', 'open', 'high', 'low', 'close', 'volume', 'tradesDone', 'volumeNotional']

    file.drop_duplicates('date', inplace=True)

    file.date = pd.to_datetime(file.date)
#     file = file.loc[:,['date', 'open']]
#     file.columns = ['Date', '%s Price'%coin]
    file.to_csv(filename)

In [178]:
# coins = ['ada', 'bch', 'bnb', 'bsv', 'btc', 'dot', 'eos', 'eth', 'link',
#        'ltc', 'xrp']

# coins = ['bsv', 'btc', 'dot', 'eos', 'eth', 'link',
#        'ltc', 'xrp']

# coins = ['xrp', 'eth']
# coins = ['xem', 'gam', 'dash', 'etc', 'xmr']
coins = ['bnb']


dates = pd.date_range(start="2017-12-15", end="2021-06-01", freq="3D")
dates_end = pd.date_range(start="2017-12-18", end="2021-06-04", freq="3D")

_counter = counter(1,0,3, 450)
error_log = []
error_tick = []

for coin in coins:
    unit = 'usd'
    tickers = coin+unit
    filename="tiingo_%s.csv"%coin

    print(filename)
    print(tickers)
    
    error = get_price(tickers, dates, dates_end, _counter)
    error_log.append(error)
    try:
        clean_data(filename)
    except:
        error_tick.append(coin)


tiingo_bnb.csv
bnbusdt
2017-12-15 2017-12-18
hourly request count: 1
Empty Json
2017-12-18 2017-12-21
hourly request count: 2
Empty Json
2017-12-21 2017-12-24
hourly request count: 3
Empty Json
2017-12-24 2017-12-27
hourly request count: 4
Empty Json
2017-12-27 2017-12-30
hourly request count: 5
Empty Json
2017-12-30 2018-01-02
hourly request count: 6
Empty Json
2018-01-02 2018-01-05
hourly request count: 7
Empty Json
2018-01-05 2018-01-08
hourly request count: 8
Empty Json
2018-01-08 2018-01-11
hourly request count: 9
Empty Json
2018-01-11 2018-01-14
hourly request count: 10
Empty Json
2018-01-14 2018-01-17
hourly request count: 11
Empty Json
2018-01-17 2018-01-20
hourly request count: 12
Empty Json
2018-01-20 2018-01-23
hourly request count: 13
Empty Json
2018-01-23 2018-01-26
hourly request count: 14
Empty Json
2018-01-26 2018-01-29
hourly request count: 15
Empty Json
2018-01-29 2018-02-01
hourly request count: 16
Empty Json
2018-02-01 2018-02-04
hourly request count: 17
Empty Json


KeyboardInterrupt: 

In [177]:
error_log

[[[('bsvusd', '2017-12-15', '2017-12-18')],
  [('bsvusd', '2017-12-18', '2017-12-21')],
  [('bsvusd', '2017-12-21', '2017-12-24')],
  [('bsvusd', '2017-12-24', '2017-12-27')],
  [('bsvusd', '2017-12-27', '2017-12-30')],
  [('bsvusd', '2017-12-30', '2018-01-02')],
  [('bsvusd', '2018-01-02', '2018-01-05')],
  [('bsvusd', '2018-01-05', '2018-01-08')],
  [('bsvusd', '2018-01-08', '2018-01-11')],
  [('bsvusd', '2018-01-11', '2018-01-14')],
  [('bsvusd', '2018-01-14', '2018-01-17')],
  [('bsvusd', '2018-01-17', '2018-01-20')],
  [('bsvusd', '2018-01-20', '2018-01-23')],
  [('bsvusd', '2018-01-23', '2018-01-26')],
  [('bsvusd', '2018-01-26', '2018-01-29')],
  [('bsvusd', '2018-01-29', '2018-02-01')],
  [('bsvusd', '2018-02-01', '2018-02-04')],
  [('bsvusd', '2018-02-04', '2018-02-07')],
  [('bsvusd', '2018-02-07', '2018-02-10')],
  [('bsvusd', '2018-02-10', '2018-02-13')],
  [('bsvusd', '2018-02-13', '2018-02-16')],
  [('bsvusd', '2018-02-16', '2018-02-19')],
  [('bsvusd', '2018-02-19', '201

In [119]:
error_tick

# bnbusdt, usdtusd

error = get_price('bnbusdt', dates, dates_end, _counter)
filename="tiingo_bnbusdt.csv"
clean_data(filename)


2017-12-15 2017-12-18
hourly request count: 451
Empty Json
2017-12-18 2017-12-21
hourly request count: 452
Empty Json
2017-12-21 2017-12-24
hourly request count: 453
Empty Json
2017-12-24 2017-12-27
hourly request count: 454
Empty Json
2017-12-27 2017-12-30
hourly request count: 455
Empty Json
2017-12-30 2018-01-02
hourly request count: 456
Empty Json
2018-01-02 2018-01-05
hourly request count: 457
Empty Json
2018-01-05 2018-01-08
hourly request count: 458
Empty Json
2018-01-08 2018-01-11
hourly request count: 459
Empty Json
2018-01-11 2018-01-14
hourly request count: 460
Empty Json
2018-01-14 2018-01-17
hourly request count: 461
Empty Json
2018-01-17 2018-01-20
hourly request count: 462
Empty Json
2018-01-20 2018-01-23
hourly request count: 463
Empty Json
2018-01-23 2018-01-26
hourly request count: 464
Empty Json
2018-01-26 2018-01-29
hourly request count: 465
Empty Json
2018-01-29 2018-02-01
hourly request count: 466
Empty Json
2018-02-01 2018-02-04
hourly request count: 467
Empty Js

hourly request count: 590
Empty Json
2019-02-08 2019-02-11
hourly request count: 591
Empty Json
2019-02-11 2019-02-14
hourly request count: 592
Empty Json
2019-02-14 2019-02-17
hourly request count: 593
Empty Json
2019-02-17 2019-02-20
hourly request count: 594
Empty Json
2019-02-20 2019-02-23
hourly request count: 595
Empty Json
2019-02-23 2019-02-26
hourly request count: 596
Empty Json
2019-02-26 2019-03-01
hourly request count: 597
Empty Json
2019-03-01 2019-03-04
hourly request count: 598
Empty Json
2019-03-04 2019-03-07
hourly request count: 599
Empty Json
2019-03-07 2019-03-10
hourly request count: 600
Empty Json
2019-03-10 2019-03-13
hourly request count: 601
Empty Json
2019-03-13 2019-03-16
hourly request count: 602
Empty Json
2019-03-16 2019-03-19
hourly request count: 603
Empty Json
2019-03-19 2019-03-22
hourly request count: 604
Empty Json
2019-03-22 2019-03-25
hourly request count: 605
Empty Json
2019-03-25 2019-03-28
hourly request count: 606
Empty Json
2019-03-28 2019-03-

hourly request count: 729
Empty Json
2020-03-31 2020-04-03
hourly request count: 730
Empty Json
2020-04-03 2020-04-06
hourly request count: 731
Empty Json
2020-04-06 2020-04-09
hourly request count: 732
Empty Json
2020-04-09 2020-04-12
hourly request count: 733
Empty Json
2020-04-12 2020-04-15
hourly request count: 734
Empty Json
2020-04-15 2020-04-18
hourly request count: 735
Empty Json
2020-04-18 2020-04-21
hourly request count: 736
Empty Json
2020-04-21 2020-04-24
hourly request count: 737
Empty Json
2020-04-24 2020-04-27
hourly request count: 738
Empty Json
2020-04-27 2020-04-30
hourly request count: 739
Empty Json
2020-04-30 2020-05-03
hourly request count: 740
Empty Json
2020-05-03 2020-05-06
hourly request count: 741
Empty Json
2020-05-06 2020-05-09
hourly request count: 742
Empty Json
2020-05-09 2020-05-12
hourly request count: 743
Empty Json
2020-05-12 2020-05-15
hourly request count: 744
Empty Json
2020-05-15 2020-05-18
hourly request count: 745
Empty Json
2020-05-18 2020-05-

hourly request count: 870
2021-05-28 2021-05-31
hourly request count: 871
2021-05-31 2021-06-03
hourly request count: 872


FileNotFoundError: [Errno 2] No such file or directory: 'tiingo_bnbusdt.csv'

In [ ]:
error = get_price('usdtusd', dates, dates_end, _counter)
filename="tiingo_usdtusd.csv"
clean_data(filename)

In [163]:
tickers = 'bnbusdt'
query = {'tickers':tickers,
         'startDate': "2021-05-04", 
         'endDate': "2021-06-04",
         'resampleFreq': '1Day'}

r = requests.get("https://api.tiingo.com/tiingo/crypto/prices", 
                 params = query,
                 headers=headers)

In [165]:
r.json()

[{'ticker': 'bnbusdt',
  'baseCurrency': 'bnb',
  'quoteCurrency': 'usdt',
  'priceData': [{'tradesDone': 2253.0,
    'date': '2021-05-12T00:00:00+00:00',
    'close': 589.53768,
    'volumeNotional': 878139.9558672,
    'high': 684.71075,
    'low': 589.53768,
    'open': 672.32682,
    'volume': 1489.54},
   {'tradesDone': 2135.0,
    'date': '2021-05-13T00:00:00+00:00',
    'close': 571.67343,
    'volumeNotional': 957604.4458587001,
    'high': 632.72353,
    'low': 535.04338,
    'open': 572.21528,
    'volume': 1675.09},
   {'tradesDone': 1048.0,
    'date': '2021-05-14T00:00:00+00:00',
    'close': 602.52213,
    'volumeNotional': 383999.40389159997,
    'high': 614.30499,
    'low': 566.96852,
    'open': 568.11593,
    'volume': 637.32},
   {'tradesDone': 889.0,
    'date': '2021-05-15T00:00:00+00:00',
    'close': 563.42771,
    'volumeNotional': 433546.35429080005,
    'high': 610.55551,
    'low': 561.17258,
    'open': 605.923,
    'volume': 769.48},
   {'tradesDone': 1408